In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import pickle as pkl
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
df = pd.read_csv("books_data/content_data.csv", index_col=0)
df = df.drop('Unnamed: 0', axis = 1)
df

,Desc,author,genre,image_link,rating,title
0,We know that power is shifting: From West to E...,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...
1,Following the success of The Accidental Billio...,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t..."
2,How to tap the power of social software and ne...,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...
3,William J. Bernstein is an American financial ...,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing
4,Amazing book. And I joined Steve Jobs and many...,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony
...,...,...,...,...,...,...
2446,"Ralph Roberts, a sus setenta años y tras la mu...",Stephen King,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,3.82,Insomnia
2447,Murder at the Vicarage marks the debut of Agat...,Agatha Christie,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Murder at the Vicarage
2448,In 1951 John Wyndham published his novel The D...,John Wyndham,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.02,The Day of the Triffids
2449,This now classic book revealed Flannery O'Conn...,Flannery O'Connor,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.22,A Good Man is Hard to Find and Other Stories


In [3]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)

In [4]:
def make_lower_case(text):
    return text.lower()

In [5]:
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

In [6]:
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [7]:
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [8]:
df['cleaned_desc'] = df['Desc'].apply(_removeNonAscii)
df['cleaned_desc'] = df.cleaned_desc.apply(func = make_lower_case)
df['cleaned_desc'] = df.cleaned_desc.apply(func = remove_stop_words)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_punctuation)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_html)

In [9]:
df['cleaned_desc'][0]

'know power shifting west east north south presidential palaces public squares formidable corporate behemoths nimble startups and slowly surely men women power merely shifting dispersing also decaying power today constrained risk losing ever before end power award winning columnist former foreign policy editor moiss nam illuminates struggle once dominant megaplayers new micropowers challenging every field human endeavor drawing provocative original research nam shows antiestablishment drive micropowers topple tyrants dislodge monopolies open remarkable new opportunities also lead chaos paralysis nam deftly covers seismic changes underway business religion education within families matters war peace examples abound walks life 1977 eighty nine countries ruled autocrats today half world s population lives democracies ceo s constrained shorter tenures predecessors modern tools war cheaper accessible make possible groups like hezbollah afford drones second half 2010 top ten hedge funds earn

In [10]:
df

,Desc,author,genre,image_link,rating,title,cleaned_desc
0,We know that power is shifting: From West to E...,Moisés Naím,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.63,The End of Power: From Boardrooms to Battlefie...,know power shifting west east north south pres...
1,Following the success of The Accidental Billio...,Blake J. Harris,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.94,"Console Wars: Sega, Nintendo, and the Battle t...",following success accidental billionaires mone...
2,How to tap the power of social software and ne...,Chris Brogan,Business,https://i.gr-assets.com/images/S/compressed.ph...,3.78,Trust Agents: Using the Web to Build Influence...,tap power social software networks build busin...
3,William J. Bernstein is an American financial ...,William J. Bernstein,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.20,The Four Pillars of Investing,william j bernstein american financial theoris...
4,Amazing book. And I joined Steve Jobs and many...,Akio Morita,Business,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Made in Japan: Akio Morita and Sony,amazing book joined steve jobs many akio morit...
...,...,...,...,...,...,...,...
2446,"Ralph Roberts, a sus setenta años y tras la mu...",Stephen King,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,3.82,Insomnia,ralph roberts sus setenta aos tras la muerte d...
2447,Murder at the Vicarage marks the debut of Agat...,Agatha Christie,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.05,Murder at the Vicarage,murder vicarage marks debut agatha christies u...
2448,In 1951 John Wyndham published his novel The D...,John Wyndham,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.02,The Day of the Triffids,1951 john wyndham published novel day triffids...
2449,This now classic book revealed Flannery O'Conn...,Flannery O'Connor,Non-Fiction,https://i.gr-assets.com/images/S/compressed.ph...,4.22,A Good Man is Hard to Find and Other Stories,classic book revealed flannery o connor one or...


In [12]:
def recommend(title):
    data = df.loc[df['title'] == title]
    data = df.loc[df['genre'] == data['genre'][0]]
    data.reset_index(level = 0, inplace = True) 
    indices = pd.Series(data.index, index = data['title'])
    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english')
    tfidf_matrix = tf.fit_transform(data['title'])
    # Calculating the similarity measures based on Cosine Similarity
    sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
    idx = indices[title]
    sig = list(enumerate(sg[idx]))
    sig = sorted(sig, key=lambda x: x[1], reverse=True)[1:6]
    for i in sig:
        print("title = ", data['title'][i[0]])
        print("author = ", data['author'][i[0]])
        print("genre = ", data['genre'][i[0]])
        print("image = ", data['image_link'][i[0]])
   

In [13]:
recommend( "The End of Power: From Boardrooms to Battlefields and Churches to States, Why Being In Charge Isn't What It Used to Be")

title =  Console Wars: Sega, Nintendo, and the Battle that Defined a Generation
author =  Blake J. Harris
genre =  Business
image =  https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1391419511l/18505802.jpg
title =  Trust Agents: Using the Web to Build Influence, Improve Reputation, and Earn Trust
author =  Chris Brogan
genre =  Business
image =  https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1347862171l/6242611.jpg
title =  The Four Pillars of Investing
author =  William J. Bernstein
genre =  Business
image =  https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1388237858l/79351.jpg
title =  Made in Japan: Akio Morita and Sony
author =  Akio Morita
genre =  Business
image =  https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1340866816l/1008101.jpg
title =  That Will Never Work: The Birth of Netflix and the Amazing Life of an Idea
author =  Marc Randolph
genre =  Business
image =  https://i.gr-assets.com/

In [ ]:
#pkl.dump(df, open('df.pkl', 'wb'))

In [ ]:
#pkl.dump(df, open('sg.pkl', 'wb'))